In [ ]:
# XXX Debug
from notebooks import *
sg.init(None, skip=[sg_load.load_search_recs])
get_search_recs.cache_clear()
search_recs = get_search_recs(cache_type=None)

In [ ]:
from notebooks import *
sg.init(None, skip=[sg_load.load_search_recs])
get_search_recs.cache_clear()
kwargss = [
    dict(refresh=True),  # Cache miss
    dict(),  # Cache hit
]
search_recss = []
for kwargs in kwargss:
    print()
    print(f'search_recs(**{kwargs})')
    search_recs = get_search_recs(**kwargs, cache_type='parquet')
    search_recss.append(search_recs)
    display(search_recs
        [['xc_id', 'feat', 'f_f', 'f_p', 'background', 'background_species']]
        [2:3].reset_index(drop=True).T
        .pipe(df_assign_first,
            size=lambda df: df.T.memory_usage(deep=True),
            type=lambda df: df[0].map(lambda x: type(x).__name__),
        )
    )

In [ ]:
# TODO Write tests
#   - Should roundtrip, should preserve category dtypes, should fail if index present
[a, b] = search_recss
pd.testing.assert_frame_equal(a, b, check_index_type=True, check_column_type=True)

# XXX Q: Is npy the whole col at once faster than npy each value separately?

In [ ]:
df = search_recs

In [ ]:
display(
    df.feat.map(lambda x: x.shape)[:3],
    df.feat[:3],
)

In [ ]:
# Slow
xs = df.feat.map(np_save_to_bytes)
display(
    xs.shape,
    len(xs),
    xs.map(len)[:3],
    xs.iloc[0][:100],
)

In [ ]:
# Fast-ish
x = np.array(list(df.feat))
b = np_save_to_bytes(x)
display(
    x.shape,
    len(b),
    b[:100],
)

In [ ]:
# Slow
(df
    .pipe(lambda df: df.assign(**{
        f'{k}.{i}': df[k].str[i]
        for k, n in {'feat': 1500}.items()
        for i in tqdm(range(n))
    }))
    [['xc_id', 'feat.0', 'feat.1', 'feat.2']]
    [:3]
)

In [ ]:
%%timeit -n1 -r3
# np.array(big_list) is fast-ish
x = np.array(list(df.feat))

In [ ]:
%%timeit -n1 -r3
# DF(big_array) is fast
DF(x).shape

In [ ]:
df0 = df[:10]
path = '/tmp/x.feather'
# df0.to_feather(path)  # XXX ArrowNotImplementedError: list<item: double>

In [ ]:
df0 = df[:10]
path = '/tmp/x.pyarrow'
df0.to_parquet(path, engine='pyarrow')
out = pd.read_parquet(path, engine='pyarrow')
pd.testing.assert_frame_equal(
    df0.pipe(df_cat_to_str),  # FIXME categories
    out.reset_index(drop=True),  # Drop Int64Index -> RangeIndex
    check_index_type=True, check_column_type=True,
)

In [ ]:
df0 = df
path = '/tmp/x.pyarrow'
%time df0.to_parquet(path, engine='pyarrow')         # 11.7s
%time out = pd.read_parquet(path, engine='pyarrow')  # 2.4s
pd.testing.assert_frame_equal(
    df0.pipe(df_cat_to_str),  # FIXME categories
    out.reset_index(drop=True),  # Drop Int64Index -> RangeIndex
    check_index_type=True, check_column_type=True,
)

# Q: What if we go wide?

In [ ]:
feat_cols = ['feat', 'f_f', 'f_p']
wide = (df
    # [:3]
    .pipe(lambda df: (
        pd.concat(axis=1, objs=[
            df.drop(columns=['feat', 'f_f', 'f_p']),
            *[
                DF(np.array(list(df[c]))).rename(columns=lambda i: f'{c}.{i}')
                for c in feat_cols
            ],
        ])
    ))
)
wide.shape

In [ ]:
# df0 = wide[:10]
df0 = wide
path = '/tmp/wide.parquet'
cat_cols = [c for c in df0 if df0[c].dtype.name == 'category']
%time df0.to_parquet(path, engine='fastparquet', compression='uncompressed')
%time out = pd.read_parquet(path, engine='fastparquet')
pd.testing.assert_frame_equal(
    df0,
    out.pipe(df_transform_cats, ordered=True, col_names=cat_cols),  # FIXME categories come out as ordered=False
    check_index_type=True, check_column_type=True,
)

In [ ]:
(out
    .pipe(df_inspect, lambda df: (df.shape,))
    .assign(**{k: list(out.filter(regex=f'^{k}\.').values) for k in feat_cols})
    .pipe(df_inspect, lambda df: (df.shape,))
    [lambda df: [c for c in df if not any(c.startswith(f'{k}.') for k in feat_cols)]]
    .pipe(df_inspect, lambda df: (df.shape,))
    # .memory_usage(deep=True).sort_index()
    [:1].T
)

In [ ]:
df[:1].T

In [ ]:
display(
    df.shape,
    wide.shape,
    wide.values.shape,
)

(35231, 58)

(35231, 3386)

(35231, 3386)

In [ ]:
# NOTE Not always correct...
df.memory_usage(deep=True).sort_values(ascending=False)[:10]

feat          426436024
id             24629926
path           24629926
basename       19019126
remarks        18617184
place          12394483
locality       10044112
license         9892276
download        8464461
place_only      8131687
dtype: int64

In [ ]:
# NOTE np.save bloats non-numeric arrays -- e.g remarks 1.8m -> 500m!
(df
    # [:10]
    .pipe(lambda df: (
        [np.save(f'/tmp/df-{k}.npy', np.array(list(df[k]))) for k in tqdm(df.columns)],  #
    ))
);

In [ ]:
# Correct results from .parquet the non-array cols and .npy the array cols...
(df
    # [:10]
    .pipe(lambda df: (
        [df[[k]].to_parquet(f'/tmp/df-{k}.parquet') for k in tqdm(df.columns) if k not in feat_cols],
        [np.save(f'/tmp/df-{k}.npy', np.array(list(df[k]))) for k in tqdm(df.columns) if k in feat_cols],
    ))
);

In [ ]:
path = ensure_dir('/tmp/save-1')
feat_cols = ['feat', 'f_f', 'f_p']
measure = lambda desc, f: timed_print(f, print=lambda x: print(f'{x} {desc}'))

# TODO TODO Save/load separate files: non_feats.parquet + feats.npy
- Finally a promising approach
- A little more complexity to manage, but not much
- Great! 1.1s load, 3.0 save
- Solution -> notebooks/api_dev_search_recs_hybrid

In [ ]:
def save(df):
    print(len(df))
    df_require_index_is_trivial(df)
    measure('non_feats', lambda: (df
        [[k for k in df if k not in feat_cols]]
        .to_parquet(f'{path}/non_feats.parquet', engine='fastparquet', compression='uncompressed')
    ))
    [measure(k, lambda: np.save(f'{path}/feat-{k}.npy', np.array(list(df[k])))) for k in df.columns if k in feat_cols]
    return df
saved = measure('save', lambda: save(search_recs
    # [:10]
))

35231

[00:00.764] non_feats

[00:01.686] feat

[00:00.638] f_f

[00:00.331] f_p

[00:03.426] save

In [ ]:
def load():
    non_feats = measure('non_feats', lambda: (
        pd.read_parquet(f'{path}/non_feats.parquet', engine='fastparquet')
    ))
    feats = {k: measure(k, lambda: np.load(f'{path}/feat-{k}.npy')) for k in feat_cols}
    df = measure('out', lambda: non_feats.assign(**{k: list(x) for k, x in feats.items()}))
    assert (df.index == pd.RangeIndex(len(df))).all()
    df = df.reset_index(drop=True)  # Drop Int64Index, restore trivial RangeIndex
    return df
loaded = measure('total', load)
display(
    saved.shape,
    loaded.shape,
    # loaded[:1].T,
)

[00:00.393] non_feats

[00:00.252] feat

[00:00.159] f_f

[00:00.121] f_p

[00:00.360] out

[00:01.387] total

(35231, 58)

(35231, 58)

In [ ]:
pd.testing.assert_frame_equal(
    (saved
        .pipe(df_reorder_cols, last=feat_cols)
    ),
    (loaded
        # TODO TODO Manually save df.columns to restore col order
        # TODO TODO Manually save df.dtypes to restore categories (and dtypes generally)
        #   - Then we win! (fingers crossed...)
        .pipe(df_reorder_cols, last=feat_cols)
        .pipe(df_transform_cats, ordered=True, col_names=[k for k in loaded if loaded[k].dtype.name == 'category'])
    ),
    check_index_type=True,
    check_column_type=True,
)